# Exploring NASA NEX-GDDP-CMIP6 Data via OpenVisus + Intake

This notebook demonstrates how to:
- Load climate data (e.g., temperature, precipitation) from NASA NEX-GDDP-CMIP6
- Subset by time, region (lat/lon), and resolution (quality)
- Visualize the result using `xarray` and `matplotlib`

The data is accessed via a custom `Intake` catalog that wraps `OpenVisus`.

Please make sure you have installed all libraries from `requirements.txt` file before proceeding.

Developed by `Aashish Panta`, `Kyo Lee` and `Valerio Pascucci` from University of Utah and NASA JPL

### Importing Libraries

In [ ]:
import os
import intake
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr


### List Available Models, Variables, Scenario, Timeranges

In [15]:
# Load catalog
cat = intake.open_catalog("https://raw.githubusercontent.com/aashishpanta0/cmip6-intake-idx/main/cmip6_catalog.yml")

# List available models & variables
print("Models:", cat.nex_gddp_cmip6.list_models())
print("Variables:", cat.nex_gddp_cmip6.list_variables())
print("Scenarios:", cat.nex_gddp_cmip6.list_scenarios())

print("Timeranges:", cat.nex_gddp_cmip6.list_timeranges())


Models: ['ACCESS-CM2', 'CanESM5', 'CESM2', 'CMCC-CM2-SR5', 'EC-Earth3', 'GFDL-ESM4', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0']
Variables: ['hurs', 'huss', 'pr', 'rlds', 'rsds', 'sfcWind', 'tas', 'tasmax', 'tasmin']
Scenarios: ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
Timeranges: {'historical': ('1950-01-01', '2014-12-31'), 'ssp126': ('2015-01-01', '2100-12-31'), 'ssp245': ('2015-01-01', '2100-12-31'), 'ssp370': ('2015-01-01', '2100-12-31'), 'ssp585': ('2015-01-01', '2100-12-31')}


### Open the catalog and set the variables

In [9]:
ds = cat.nex_gddp_cmip6
ds=ds(model="ACCESS-CM2", 
                        variable="tasmax", 
                        scenario="historical", 
                        timestamp="2010-10-15",
                        )


In [10]:
ds.read()

<xarray.DataArray (lat: 600, lon: 1440)> Size: 3MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) float64 5kB -59.88 -59.63 -59.38 -59.13 ... 89.25 89.5 89.75
  * lon      (lon) float64 12kB 0.125 0.3749 0.6248 0.8747 ... 359.3 359.5 359.8

In [11]:
# --- User selections ---
SELECTED_MODEL    = "EC-Earth3"
SELECTED_VARIABLE = "tasmax" 
SELECTED_SCENARIO = "historical"
START_DATE = "2010-12-30"
END_DATE   = "2010-12-31"

In [12]:
import warnings
warnings.filterwarnings("ignore")

import intake

CATALOG_URL = "https://raw.githubusercontent.com/aashishpanta0/cmip6-intake-idx/main/cmip6_catalog.yml"
cat = intake.open_catalog(CATALOG_URL)
factory = cat.nex_gddp_cmip6

def _to_list(obj):
    if obj is None:
        return []
    if isinstance(obj, (list, tuple)):
        return list(obj)
    if hasattr(obj, "tolist"):
        try:
            return list(obj.tolist())
        except Exception:
            pass
    if isinstance(obj, dict):
        return list(obj.keys())
    try:
        return list(obj)
    except Exception:
        return [str(obj)]

MODELS    = _to_list(factory.list_models())
VARIABLES = _to_list(factory.list_variables())
SCENARIOS = _to_list(factory.list_scenarios())

print("Available models   :", MODELS[:5], "... total", len(MODELS))
print("Available variables:", VARIABLES)
print("Available scenarios:", SCENARIOS)

print("Using:")
print(" Model   :", SELECTED_MODEL)
print(" Variable:", SELECTED_VARIABLE)
print(" Scenario:", SELECTED_SCENARIO)
print(" Period  :", START_DATE, "to", END_DATE)
src = factory(model=SELECTED_MODEL,
              variable=SELECTED_VARIABLE,
              scenario=SELECTED_SCENARIO,
              start_date=START_DATE,
              end_date=END_DATE)



Available models   : ['ACCESS-CM2', 'CanESM5', 'CESM2', 'CMCC-CM2-SR5', 'EC-Earth3'] ... total 11
Available variables: ['hurs', 'huss', 'pr', 'rlds', 'rsds', 'sfcWind', 'tas', 'tasmax', 'tasmin']
Available scenarios: ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
Using:
 Model   : EC-Earth3
 Variable: tasmax
 Scenario: historical
 Period  : 2010-12-30 to 2010-12-31


### Parameters manipulation
quality: Change the resolution of data. 0 is full resolution, -2 is 1/4th, -4 is 1/8th and so on.

lat_range, lon_range: Subset your region of interest

In [13]:
ds = cat.nex_gddp_cmip6(model="ACCESS-CM2", 
                        variable="tas", 
                        scenario="ssp245", 
                        timestamp="2015-10-15",
                        quality=-2,
                        lat_range=(0, 40),
                        lon_range=(0, 120),
                        )

ds

nex_gddp_cmip6:
  args:
    cached: false
    lat_range: !!python/tuple
    - 0
    - 40
    lon_range: !!python/tuple
    - 0
    - 120
    model: ACCESS-CM2
    quality: -2
    scenario: ssp245
    timestamp: '2015-10-15'
    variable: tas
  description: NEX-GDDP-CMIP6 reader
  driver: intake_nexgddp.catalog.NexGDDPCatalog
  metadata:
    catalog_dir: https://raw.githubusercontent.com/aashishpanta0/cmip6-intake-idx/main


### Plot using default xarray tool

In [ ]:
ds.read().plot()

### Reading intake as xarray

In [ ]:
xr_ds=ds.read()
xr_ds

### Plot using Matplotlib

In [ ]:
data=xr_ds.values
plt.imshow(data, cmap='viridis',origin='lower')

### Select Time Ranges
Specify start date and end date

In [ ]:
da = cat.nex_gddp_cmip6(
    model="ACCESS-CM2",
    variable="tas",
    scenario="ssp245",
    start_date="2015-10-10",
    end_date="2015-10-20",
    quality=-4,
).read()
print(da.dims) 
print(da.time.values[:3])


In [ ]:
da.plot.imshow(col="time", col_wrap=3, cmap="viridis", aspect=1.5, vmin=270, vmax=310)

### Select Specific Dates

Pass your dates of interest as timestamps

In [ ]:
dates = ["2015-10-10","2015-10-12","2015-10-15"]
da = cat.nex_gddp_cmip6(
    model="ACCESS-CM2",
    variable="hurs",
    scenario="ssp245",
    timestamps=dates,
    quality=0,
    lat_range=(0, 40),
    lon_range=(0, 120),
).read()


In [ ]:
da.plot.imshow(col="time", col_wrap=3, cmap="coolwarm", aspect=1.5)

Please contact [Aashish Panta](mailto:aashishpanta0@gmail.com) if you have any concerns or questions.
